In [2]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`


In [3]:
import Random
rng = Random.MersenneTwister();

In [85]:
n = 4
ω = 0.25
ham = HOCoulomb(ω, shielding = 0.25);
#ham = HarmonicOscillator(ω)

l = 10
basis = SpinBasis(HOBasis(l, ω))
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, ham);

mixer = Alpha(0.5)
rhf = RHF(system, mixer);
rhf = compute_ground_state!(rhf, max_iter = 1000, tol = 1e-10, verbose=0);
energy(rhf)

4.466761475079295

In [86]:
slater = Slater(rhf);

In [87]:
function energy_opt(wf, optimizer)
    ham = HOCoulomb(ω, shielding = 0.25);

    metro = Importance(equils = 1e4, samples = 1e5, step = 0.01);
    t = @elapsed wf, grad_results = optimize(wf, ham, metro, optimizer, nthreads = 12, verbose = true);
    println("\nGradient time: $(t) s")
    return wf
end

function energy_block(wf)
    ham = HOCoulomb(ω, shielding = 0.25);
    samples_b = 2^22
    metro_b = Importance(equils = 1e6, samples = samples_b, step = 0.01);
    t = @elapsed result = blocking(wf, ham, metro_b, nthreads = 12);
    digits = Integer(-floor(log10(result.E_err)))
    print("$(round(result.E, digits=digits)) ± $(round(result.E_err, digits=digits)), $(round(result.E - 3.7871, digits=digits))")
    println("\nBlocking time: $(t) s")
end

energy_block (generic function with 1 method)

In [89]:
nn = NeuralNetwork([Dense(32), Tanh, Dense(8), Tanh, Dense(1), Exp], n=n, rng = rng);
wf = SlaterNN(slater, nn);
optimizer = ADAM(wf, lr = 0.001, max_iter = 50, tol = 1e-6);
wf = energy_opt(wf, optimizer);
energy_block(wf);

E = 4.615317 iter = 50/50                                      
Gradient time: 7.8776068 s
5.1 ± 0.3, 1.3
Blocking time: 7.9691538 s


In [82]:
nn = NeuralNetwork([Dense(32), Tanh, Dense(32), Tanh, Dense(1), Exp], n=n, rng = rng);
wf = SlaterNN(slater, nn);
optimizer = ADAM(wf, lr = 0.01, max_iter = 50, tol = 1e-6);
#optimizer = GradientDescent(lr = 0.01, max_iter = 50, tol = 1e-6);
wf = energy_opt(wf, optimizer);
#energy_block(wf);

E = 1.0 iter = 14/50                                      
Convergence reached, final norm of gradient was 9.930365308402076e-7

Gradient time: 3.6214994 s


In [70]:
energy_block(wf);

11.0 ± 2.0, 7.0
Blocking time: 12.4945917 s


In [88]:
ccsd = CCSD(System(rhf))
compute_ground_state!(ccsd, max_iter = 100, tol = 1e-6, verbose = 3);

Iteration 18: E = 3.798030950625483                            Iteration 19: E = 3.798031243749311


In [59]:
(4.7 - 3.798030950625483       ) / 3.798030950625483   

0.2374833331007941